# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático

**Antes de empezar:**
- [Instalar spaCy y el modelo de lenguaje con el que van a trabajar](https://spacy.io/models#quickstart)

**Consideraciones:**
- Se evalúa el estilo y prolijidad del código.
- Se permite hacer trabajo extra, siempre y cuando las actividades básicas estén resueltas.

**Recomendación:**
- Hay muchos ejemplos de código en internet, no se sientan obligados a implementar todo desde cero.

### Librerías

In [1]:
!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk
!pip3 install sklearn
!pip3 install lyricsgenius

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

In [2]:
#-------------Calculos Generales----------------------------
    
def loadTrackFeatures(features_track, tracks_id):
    
    features = sp.audio_features(tracks_id)
    for feature in features:
        for column_name in features_track.keys():           
            features_track.get(column_name).append(feature.get(column_name))
    return pd.DataFrame(features_track)
            

def loadTracksFromPlayListItems(tracks_dict, playlist_items):
    
    for item in playlist_items:
        track = item.get('track')
        for column_name in tracks_dict.keys():            
            if column_name == 'artists_name':
                artist = getArtistsNameFromArtistObjectList( track.get('album').get('artists') );
                tracks_dict['artists_name'].append(artist)
            else: 
                tracks_dict.get(column_name).append(track.get(column_name))
    return pd.DataFrame(playlist_items)

def getArtistsNameFromArtistObjectList( artistObjectList ):
  artist_name_list = []
  for artist in artistObjectList:
    artist_name_list.append(artist.get('name'))
  return artist_name_list

def getPlaylistDataFrame(playlist_id):
    offset = 0
    total_tracks = None
    artist_tracks_dict = {'artist_id':[],'artist_name': [], 'genre':[]}
    features_tracks_dict = {'id': [], 'key':[], 'mode':[], 'time_signature':[], 'tempo':[], 
                       'acousticness':[], 'danceability':[], 'energy':[], 'instrumentalness':[], 'valence':[] }
    tracks_dict = {'id': [],'name': [], 'artists_name':[] } 

    while True:        
        tracks = sp.playlist_tracks(playlist_id, offset=offset, additional_types=('track',))  
        playlist_items = tracks['items']
        limit = tracks.get('limit')

        loadTracksFromPlayListItems(tracks_dict, playlist_items)    
        loadTrackFeatures(features_tracks_dict, tracks_id=tracks_dict.get('id')[offset:offset + limit])

        total_tracks = tracks.get('total') if (total_tracks is None) else total_tracks
        offset = tracks.get('offset') + limit

        if offset > total_tracks:
            break
    df_tracks = pd.DataFrame(tracks_dict)
    df_features_tracks = pd.DataFrame(features_tracks_dict)
    data_set = pd.merge(df_tracks, df_features_tracks, how='inner', on='id')
    return data_set    

def display_markdown(*args, **kwargs):
    return display(Markdown(*args, **kwargs))


### Dependencias y acceso a APIs

In [39]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
from musixmatch import Musixmatch

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

#nlp = spacy.load("en_core_web_sm") # completar con el modelo que van a utilizar

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
musixmatch = Musixmatch('1aa5272f4402bf2f082ad2f3958c2c62') # se puede reemplazar por otra API si da mejores resultados

In [54]:
mentoria_playlist_id = '2IuD0qZb14cji5y52crdsO'
df_mentoria =  getPlaylistDataFrame(mentoria_playlist_id)
df_mentoria

,id,name,artists_name,key,mode,time_signature,tempo,acousticness,danceability,energy,instrumentalness,valence
0,7j9DYPyCuvSAtPcevpAkzb,Desafío,[Arca],1,0,5,161.092,0.486,0.161,0.482,0.409000,0.0926
1,1cwTMSQeMaA9fVKEF1iWeD,Anoche,[Arca],10,0,5,80.793,0.570,0.230,0.434,0.000000,0.0834
2,0aL27vskbMpwsMGUkHm3Zf,Sin Rumbo,[Arca],9,1,3,124.835,0.874,0.289,0.280,0.004430,0.0391
3,2kfSFdq2h0xLXq01em1zc7,La Gata Bajo la Lluvia,[Rocío Dúrcal],7,1,4,88.140,0.723,0.499,0.648,0.000000,0.4640
4,5ySxlyvySBhIEvoO2xx7uT,Querida,[Juan Gabriel],2,1,4,89.089,0.376,0.528,0.383,0.000000,0.4600
...,...,...,...,...,...,...,...,...,...,...,...,...
1225,1o7e4Tadn4o6nPvycwIP0C,En Libertad,[Rodrigo],5,0,4,150.167,0.236,0.684,0.802,0.000012,0.8390
1226,5AZwGCyE9tIzFYXpqyrKHm,"Amante Tu, Amante El",[Rodrigo],4,0,4,160.612,0.541,0.541,0.734,0.000000,0.7040
1227,6Lkh9lukUEPBTZ1gewNT0m,Todo Me Lleva a Ti,[Rodrigo],7,1,4,96.312,0.157,0.600,0.897,0.000610,0.7560
1228,4sUEtsaCvQVIkzMksupERW,La Marcha De La Bronca,[Pedro Y Pablo],9,0,4,119.200,0.696,0.423,0.593,0.000000,0.5210


In [ ]:
df_mentoria

In [53]:
import lyricsgenius
import json

genius_token = "upzErEoYU9BAcEOF0WUIQ-Rw_fxFYrU6QeqVhlHXUEc-7VI1jxZvQNiiTGDigvpk"
csv_folder_path = './TP_3_CSV_Files/{}'
#genius = lyricsgenius.Genius(genius_token)
genius = lyricsgenius.Genius("upzErEoYU9BAcEOF0WUIQ-Rw_fxFYrU6QeqVhlHXUEc-7VI1jxZvQNiiTGDigvpk")

#song = genius.search_song("Desafío", "Arca")
#print(song)
#jsonStr = json.dumps(song.__dict__)
#test_dict = song.__dict__
#print(song.get("_body").get("primary_artist").get("name"))
#print(song.artist)
#print(song.title)

file_name = 'Test_track' + '_1.csv'
test_path = csv_folder_path.format(file_name)
df_test = pd.read_csv(test_path, sep='#')

test =  df_test[df_test.id == '7j9DYPyCuvSAtPcevpAkzb' ]
test.empty
df_test


,id,name,lyric
0,7j9DYPyCuvSAtPcevpAkzb,Desafío,NaN
1,1cwTMSQeMaA9fVKEF1iWeD,Anoche,"[Letra de ""Anoche""]\n\n[Coro 1]\nAnoche te soñ..."
2,0aL27vskbMpwsMGUkHm3Zf,Sin Rumbo,"[Letra de ""Sin Rumbo""]\n\n[Intro]\nGirando en ..."
3,2kfSFdq2h0xLXq01em1zc7,La Gata Bajo la Lluvia,NaN
4,5ySxlyvySBhIEvoO2xx7uT,Querida,"[Letra de ""Querida""]\n\n[Verso 1]\nQuerida, ca..."
...,...,...,...
57,7zM2vaJMUVlwffYmEbXCsO,"Movidito, Movidito",NaN
58,2sb31nQbx4QnFD9FQv2D28,La Ley Y La Trampa,NaN
59,6DEZMZwxrSPDWssNBQbIS8,Testigo,NaN
60,24bl7udyFxIGgMVxTKzKJP,No Regreso,Ay que yo puedo comer\nAy que yo no puedo dorm...


In [52]:

tracks_lyric_dict = {'id': [],'name': [], 'lyric':[] } 
try:
    for index, row in df_mentoria.iterrows():
        track_name = row.get("name")
        artists = row.get("artists_name")  

        test =  df_test[df_test.id == row.get('id')]
        if test.empty:
            song = genius.search_song(track_name, artists[0])

            tracks_lyric_dict.get("id").append(row.get("id"))
            tracks_lyric_dict.get("name").append(track_name)
            lyric = None
            if song and song.artist ==  artists[0] and song.title == track_name:
                lyric = song.lyrics
            tracks_lyric_dict.get("lyric").append(lyric)
            print(index)
            if index == 100 :
                break
except:
    print("Something else went wrong")
    
df_temporal = pd.DataFrame(tracks_lyric_dict) 
df_append = df_test.append(df_temporal)
file_name = 'Test_track' + '_1.csv'
df_append.to_csv(csv_folder_path.format(file_name),  encoding='utf-8', index=False, sep= '#')
df_append



Searching for "Llegó Tu Papi" by Sabroso...
Specified song does not have a valid URL with lyrics. Rejecting.
56
Searching for "Movidito, Movidito" by Sebastian...
Specified song does not have a valid URL with lyrics. Rejecting.
57
Searching for "La Ley Y La Trampa" by Chaqueño Palavecino...
Specified song does not have a valid URL with lyrics. Rejecting.
58
Searching for "Testigo" by Combo Chimbita...
Specified song does not have a valid URL with lyrics. Rejecting.
59
Searching for "No Regreso" by Combo Chimbita...
Done.
60
Searching for "Cómo Te Voy A Olvidar" by Los Ángeles Azules...
Specified song does not have a valid URL with lyrics. Rejecting.
61
Searching for "Más Que Tu Amigo" by Marco Antonio Solís...
Something else went wrong


,id,name,lyric
0,7j9DYPyCuvSAtPcevpAkzb,Desafío,NaN
1,1cwTMSQeMaA9fVKEF1iWeD,Anoche,"[Letra de ""Anoche""]\n\n[Coro 1]\nAnoche te soñ..."
2,0aL27vskbMpwsMGUkHm3Zf,Sin Rumbo,"[Letra de ""Sin Rumbo""]\n\n[Intro]\nGirando en ..."
3,2kfSFdq2h0xLXq01em1zc7,La Gata Bajo la Lluvia,NaN
4,5ySxlyvySBhIEvoO2xx7uT,Querida,"[Letra de ""Querida""]\n\n[Verso 1]\nQuerida, ca..."
...,...,...,...
1,7zM2vaJMUVlwffYmEbXCsO,"Movidito, Movidito",None
2,2sb31nQbx4QnFD9FQv2D28,La Ley Y La Trampa,None
3,6DEZMZwxrSPDWssNBQbIS8,Testigo,None
4,24bl7udyFxIGgMVxTKzKJP,No Regreso,Ay que yo puedo comer\nAy que yo no puedo dorm...


### 1) Recopilar los datos obtenidos en los prácticos anteriores

Para esta parte consideraremos [la playlist colaborativa de la mentoría](https://open.spotify.com/playlist/2IuD0qZb14cji5y52crdsO?si=nfHRPDquQRyotEcXc4tG7Q), de esta obtendremos:
- Las features del audio de las canciones
- Las features textuales de sus letras

Además es necesario aplicar el mismo preprocesamiento que aplicamos en los prácticos anteriores para ambos tipos de features (el preprocesamiento del p1 a las features de audio y el de p2 al de features textuales) y obtener el género de cada canción, que en caso de ser más de uno para una canción el equipo deberá discutir una estrategia para estos casos y comentarla en el informe.

Luego, se separará al dataset resultante en **X** e **y**, donde:
- X es el conjunto de features
- y es la etiqueta, en este caso el género de la canción, que deberá ser codificado en valores del tipo **int**

Por último, se dividirá a estos dos conjuntos en los splits **train** y **test**

**Recomendaciones:**
- Obtener las features por separado y hacer un join de los datasets.
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)
- Si usan features categóricas, ENCODEARLAS!

### 2) Elegir tres modelos de clasificadores multiclase

Aquí escogeremos tres modelos diferentes y luego compararemos su rendimiento para esta tarea. El procedimiento será el siguiente:
- Inicializar los modelos
- Entrenarlos usando el split **train** de los datos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 3) Informe: Comparar el rendimiento de los modelos

Una vez entrenados los tres modelos, compararemos su rendimiento:
- Correr los modelos usando el split **test**
- Obtener el reporte de clasificación y la matriz de confusión para cada modelo
- Graficar llevando a 2 dimensiones nuestro split **test** pintando con colores diferentes según la etiqueta correspondiente.
- Graficar de manera similar los resultados obtenidos con cada clasificador y sobre esto la función de clasificación obtenida.
- Guardar los modelos usando **pickle**
- Discutir los resultados obtenidos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 4) Tareas adicionales:

Estas tareas servirán para extrapolar un poco el trabajo básico, y también sumarán puntos extra. Deben elegir una o más de las siguientes:
-  Análisis sobre el balance de clases del dataset, balanceo usando **subsampling** u **oversampling** y comparación de resultados vs el modelo básico
- Optimización de hiperparámetros y comparación de resultados vs el modelo básico
- Graficar importancia de features
- Graficar correlación de features

**Recomendación:**
- Hacer varias ahora puede ahorrarles tiempo en el futuro